In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.20.0-dev20250405


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-04-07 14:57:23--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.4’

train-data.tsv.4    100%[===================>] 349.84K  --.-KB/s    in 0.02s   

2025-04-07 14:57:23 (13.8 MB/s) - ‘train-data.tsv.4’ saved [358233/358233]

--2025-04-07 14:57:24--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.4’

valid-data.tsv.4    100%[====

In [ ]:
df_train = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'text'])
df_test =  pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'text'])

In [ ]:
# Concatenar todos los textos del conjunto de entrenamiento para obtener el vocabulario completo (a nivel de caracteres)
corpus = " ".join(df_train["text"].tolist())
vocab = sorted(set(corpus))

# Crear mapeo: de carácter a índice y viceversa
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
    """Convierte un texto a una secuencia de enteros usando el mapeo definido."""
    return np.array([char2idx[c] for c in text if c in char2idx])

# Convertir cada texto del DataFrame a secuencia de enteros
df_train["int_sequence"] = df_train["text"].apply(text_to_int)
df_test["int_sequence"] = df_test["text"].apply(text_to_int)

# Convertir las secuencias (que están en cada celda del DataFrame) a listas para poder usar pad_sequences
train_sequences = df_train["int_sequence"].tolist()
test_sequences = df_test["int_sequence"].tolist()
# Dividir los labels en variables separadas
train_labels = df_train["label"].values
test_labels = df_test["label"].values

# Convertir los labels: se asigna 1 a "spam" y 0 a los demás (por ejemplo, "ham")
train_labels = np.where(train_labels == "spam", 1, 0)
test_labels = np.where(test_labels == "spam", 1, 0)

In [ ]:
max_len = 250
train_pad = keras.preprocessing.sequence.pad_sequences(train_sequences, max_len)
test_pad = keras.preprocessing.sequence.pad_sequences(test_sequences, max_len)

In [ ]:
vocab_size = len(vocab)
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, 64),
    keras.layers.LSTM(32),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.000778)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['acc'])

In [ ]:
model.fit(train_pad, train_labels, epochs=10, shuffle=True)

Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - acc: 0.8621 - loss: 0.3888
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - acc: 0.9675 - loss: 0.1261
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - acc: 0.9723 - loss: 0.1086
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - acc: 0.9735 - loss: 0.0927
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - acc: 0.9775 - loss: 0.0834
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - acc: 0.9829 - loss: 0.0653
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - acc: 0.9809 - loss: 0.0724
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - acc: 0.9850 - loss: 0.0608
Epoch 9/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - acc: 0.9827 - loss: 0.0711
Epoch 10/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - acc: 0.9821 - loss: 0.0615


In [ ]:
model.evaluate(test_pad, test_labels)

44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - acc: 0.9850 - loss: 0.0545


[0.05625052750110626, 0.9841954112052917]

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  prediction, label = [], None
  encoded_text = text_to_int(pred_text)
  text_pad = keras.preprocessing.sequence.pad_sequences([encoded_text], max_len)
  probability = model.predict(text_pad)
  prediction.append(probability[0][0].item())

  if probability[0][0] > 0.01:
    label = 'spam'
  else:
    label = 'ham'
  prediction.append(label)

  return (prediction)

pred_text = "our new mobile video service is live. just install on your phone to start watching."
prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
[0.019829999655485153, 'spam']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
You passed the challenge. Great job!
